<a href="https://colab.research.google.com/github/maya-papaya/ads1-cervical-cancer-analysis/blob/main/First%20Semester%20Project%3A%20Creating%20Relational%20Database%20(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Relational Database

In this notebook, I created a relational database consisting of the datasets I had cleaned/preprocessed in `First Semester Project: Data Cleaning & Preprocessing (2).ipynb`. When assigning primary keys, I occasionally used two columns in order to ensure uniqueness; the code for this process was sourced from https://tableplus.com/blog/2018/04/sqlite-specify-primary-key-on-multiple-columns.html#:~:text=How%20to%20specify%20primary%20key,KEY%20(column1%2C%20column2)%20)%3B.

In [ ]:
# SETTING UP DRIVE AND MODULES
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir("/content/drive/My Drive/ADS_Maya_Reddy/projects/disease_project_1/prepped_datasets/")

Mounted at /content/drive


In [ ]:
# LOADING DATASETS
data = pd.read_csv('data.csv')
pap = pd.read_csv('pap.csv')
hpv = pd.read_csv('hpv.csv')
adolescent = pd.read_csv('adolescent.csv')
demo = pd.read_csv('demo.csv')

In [ ]:
# CREATING DATABASE TABLES
import sqlite3
connection = sqlite3.connect('cervical_cancer.db')
cursor = connection.cursor()

# CREATE THE 'data' TABLE
cursor.execute('''
CREATE TABLE IF NOT EXISTS data (
    state TEXT NOT NULL,
    measure TEXT NOT NULL,
    metric TEXT NOT NULL,
    year INTEGER NOT NULL,
    val FLOAT NOT NULL,
    upper FLOAT NOT NULL,
    lower FLOAT NOT NULL,
    PRIMARY KEY (state, year)
)''')
data.to_sql('data', connection, if_exists='replace', index=False)

# CREATE THE 'pap' TABLE
cursor.execute('''
CREATE TABLE IF NOT EXISTS pap (
    city TEXT NOT NULL,
    pop_2010, INTEGER NOT NULL,
    val FLOAT NOT NULL,
    lower FLOAT NOT NULL,
    upper FLOAT NOT NULL,
    year INTEGER NOT NULL,
    FOREIGN KEY (state) REFERENCES data(state),
    PRIMARY KEY (city, year)
)''')
pap.to_sql('pap', connection, if_exists='replace', index=False)

# CREATE THE 'hpv' TABLE
cursor.execute('''
CREATE TABLE IF NOT EXISTS hpv (
    year PRIMARY KEY,
    cohort_size FLOAT NOT NULL,
    current_cov FLOAT NOT NULL,
    curr_vacc_cohort_size FLOAT NOT NULL,
    future_cov FLOAT NOT NULL,
    future_vacc_cohort_size FLOAT NOT NULL,
    curr_cc_prev FLOAT NOT NULL,
    curr_mort_prev FLOAT NOT NULL,
    curr_cost FLOAT NOT NULL,
    curr_cost_prev FLOAT NOT NULL,
    proj_cc_prev FLOAT NOT NULL,
    proj_mort_prev FLOAT NOT NULL,
    proj_cost FLOAT NOT NULL,
    current_net_cost FLOAT NOT NULL,
    country TEXT NOT NULL
)''')
hpv.to_sql('hpv', connection, if_exists='replace', index=False)

# CREATE THE 'demo' TABLE
cursor.execute('''
CREATE TABLE IF NOT EXISTS demo (
    state TEXT NOT NULL,
    year TEXT NOT NULL,
    dimension_type TEXT NOT NULL,
    dimension_val TEXT NOT NULL,
    val FLOAT NOT NULL,
    FOREIGN KEY (state) REFERENCES data(state),
    PRIMARY KEY (state, year, dimension_type, dimension_val)
)''')
demo.to_sql('demo', connection, if_exists='replace', index=False)

# CREATE THE 'adolescent' TABLE
cursor.execute('''
CREATE TABLE IF NOT EXISTS adolescent (
    state TEXT NOT NULL,
    year TEXT NOT NULL,
    dose TEXT NOT NULL,
    dimension_type TEXT NOT NULL,
    dimension_val TEXT NOT NULL,
    val FLOAT NOT NULL,
    lower FLOAT NOT NULL,
    upper FLOAT NOT NULL,
    FOREIGN KEY (state) REFERENCES data(state),
    PRIMARY KEY (state, year, dose)
)''')
adolescent.to_sql('adolescent', connection, if_exists='replace', index=False)

# COMMIT TO DATABASE
connection.commit()

In [ ]:
# CHECKING DATABASE
pd.read_sql('SELECT * FROM data', connection)

,state,year,mort_count,mort_rate,inc_count,inc_rate
0,Maine,1990,29.106219,4.535542,140.611927,21.911171
1,Minnesota,1990,72.526393,3.191742,388.588691,17.101014
2,Colorado,1990,55.543617,3.269754,320.913420,18.891604
3,Pennsylvania,1990,361.068921,5.739195,1714.406730,27.250516
4,Mississippi,1990,80.440675,5.877213,361.823577,26.435808
...,...,...,...,...,...,...
1627,District of Columbia,2020,15.947269,4.384439,55.139795,15.159779
1628,Oregon,2020,74.350922,3.461421,293.773819,13.676696
1629,Michigan,2020,209.653898,4.081703,737.460064,14.357440
1630,South Dakota,2020,14.422204,3.271999,55.710635,12.639200


In [ ]:
# CLOSING CONNECTION
connection.close()